In [1]:
import numpy as np
from genetic_selection import GeneticSelectionCV
from model_training import get_X_Y
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import global_config as cfg

csv_path = cfg.padel_csvfilepath
X, blood_y, brain_y, ratio_y, SMILES = get_X_Y(csv_path)

y = ratio_y
model_type = cfg.model_type
param_name = 'blood_params'
params = cfg.model_params.get(model_type)
model = XGBRegressor()

selector = GeneticSelectionCV(
    model,
    cv=5,
    verbose=1,
    scoring="r2",
    max_features=50,
    n_population=400,
    crossover_proba=0.5,
    mutation_proba=0.2,
    n_generations=30,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.05,
    tournament_size=3,
    n_gen_no_change=10,
    caching=True,
    n_jobs=-1,
)
selector = selector.fit(X, y)



c:\Users\80473\anaconda3\envs\BBB\lib\site-packages\ipykernel_launcher.py:10: DtypeWarning: Columns (2401,2402,2444,2448,2450,2461,2467,2471,2520,2521,2563,2567,2569,2580,2586,2590,2761,2762) have mixed types.Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.


Selecting features with genetic algorithm.
gen	nevals	avg                                  	std                                  	min                                     	max                                  
0  	200   	[-344.161798   25.88      600.756468]	[ 382.768289   14.304041  698.310562]	[-1848.896794     1.           1.756379]	[  -1.480471   50.       3512.943684]
1  	133   	[-2121.981778   118.955     2208.154053]	[ 3944.785867   189.167156  3915.31447 ]	[-10000.            2.            1.291411]	[   -1.182307   575.       10000.      ]
2  	120   	[-2208.432671   127.16      2281.872199]	[ 4023.79869    199.496602  3997.429916]	[-10000.            2.            1.291411]	[   -0.857396   954.       10000.      ]
3  	116   	[-2439.617274   140.4       2466.651223]	[ 4250.841235   199.364892  4240.514382]	[-10000.           12.            1.064477]	[   -0.857396   665.       10000.      ]
4  	128   	[-1688.422881   108.625     1718.415917]	[ 3696.77152    176.001547  3688.386152

In [2]:
print(selector.support_)
print(selector.n_features_)
fea = []
idx = 0
for i in selector.support_:
    if i:
        fea.append(idx)
    idx = idx + 1
print(fea)

f = open('./ga_output.txt', 'w')
f.write(str(selector.support_))
f.write("\n")
f.write(str(fea))
f.close()

[False False False ... False False False]
24
[23, 301, 1115, 1370, 1575, 2049, 2933, 3013, 3143, 3342, 3468, 3737, 4089, 4569, 5793, 5891, 6448, 7099, 7552, 7796, 7874, 8339, 9416, 9491]


In [3]:
X_s = X.iloc[:, selector.support_]
X_train, X_val, y_train, y_val = train_test_split(X_s, y, test_size=0.2)
model.fit(X_train, y_train)
pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, pred))
r2 = r2_score(y_val, pred)
print("RMSE: ", rmse)
print("R2: ", r2)


RMSE:  1.4492767157913589
R2:  -13.65605953917163
